In [95]:
## pc_op[1:0] = instr [25:24]  --> Wybór operacji skoku 
## alu_op[1:0] = instr [21:20]  --> Wybór operacji ALU
## rx_op[2:0] = instr [18:16]  --> Multiplekser rejestrów nr 1
## imm_op = instr [15]  --> Wybór do ALU (imm lub ry)
## ry_op[2:0] = instr [14:12]  --> Multiplekser rejestrów nr 2
## rd_op = instr[11]  --> Wybór źródła zapisu ALU/pamięć
## d_op[2:0] = instr[10:8]  --> Wybór rejestru do zapisu
## imm[7:0] = instr[7:0]  --> Wartość imm

In [96]:
# liczba argumentów
# nop - 0
# jump, jumpi - 1
# jz, jnz, load, loadi - 2
# add, addi, and, andi - 3

# Odczyt z pliku
f = open("tekst2.txt")
lista = []
for line in f:
    for i in line.split(","):
        for x in i.split(" "):
            if '\n' in x:
                lista.append(x[:-1])
            elif x != '':
                lista.append(x)

new_lista = [i for i in lista if i != '']
d = []
cnt = 0
idx = 0
idx_d = -1

for com in new_lista:
    if com in ['movi','mov','nop','jump','jumpi','jz','jnz','load','loadi','add','addi','and','andi'] and cnt == 0:
        if com == 'nop':
            cnt = 0
        elif com in ['jump','jumpi']:
            cnt = 1
        elif com in ['jz','jnz','load','loadi','movi','mov']:
            cnt = 2
        elif com in ['add','addi','and','andi']:
            cnt = 3
        d.append([com])
        idx_d +=1
    else:
        d[idx_d].append(com)
        cnt -= 1
print(d)

[['movi', 'R0', '0'], ['movi', 'R1', '4'], ['addi', 'R0', 'R0', '1'], ['and', 'R2', 'R0', 'R1'], ['jz', 'R2', '2'], ['movi', 'R3', '1']]


In [97]:
# Wybór do ALU - imm
def val2imm(command):
    ans = ""
    if command in ["R0","R1","R2","R3","R4","R5","R6","R7"]:
        ans = "00000000"
    else:
        if 'x' in command:
            bine = str(bin(int(command, base=16))[2:])
        else:
            bine = str(bin(int(command,base=10))[2:])

        if len(bine) < 8:
            ans = (8 - len(bine))* '0' + bine
        else:
            ans = bine
    return ans

val2imm('200')

'11001000'

In [98]:
# Multiplekser rejestrów
def reg2bin (register):
    ans = ""
    # Odczyt z rejestru
    if register == "R0":
        ans = "000"
    elif register == "R1":
        ans = "001"
    elif register == "R2":
        ans = "010"
    elif register == "R3":
        ans = "011"
    elif register == "R4":
        ans = "100"
    elif register == "R5":
        ans = "101"
    elif register == "R6":
        ans = "110"
    elif register == "R7":
        ans = "111"
    else:
        raise Exception("Wrong input")
    return ans

In [99]:
def parse(list_of_commands):
    ans = ''
    lista = []
    for command in list_of_commands:
        if len(command) == 1:
            # Tylko przy nop ---> ???

            print(command)
            pc_op = "00" 
            alu_op = "11"
            rx_op = "110"
            imm_op = "0"
            ry_op = "110"
            rd_op = "0"
            d_op = "110"
            imm = "00000000"

            
        elif len(command) == 2:
            # jump, jumpi

            print(command)
            pc_op = "01" 
            ry_op = "110"
            rd_op = "0"
            d_op = "110"

            if command[0] == 'jump':
                alu_op = "01"
                rx_op = reg2bin(command[1])
                imm_op = "0"
                imm = "00000000"

            elif command[0] == 'jumpi':
                alu_op = "11"
                rx_op = "110"
                imm_op = "1"
                imm = val2imm(command[1])
          
        elif len(command) == 3:
        # jz, jnz, load, loadi, mov, movi
            
            print(command)
            if command[0] == 'jz':
                pc_op = "10"
                alu_op = "01" # -------> ??
                rx_op = reg2bin(command[1])
                imm_op = "1"
                ry_op = "110"
                rd_op = "0"
                d_op = "110"
                imm = val2imm(command[2])

            elif command[0] == 'jnz':
                pc_op = "11"
                alu_op = "01"
                rx_op = reg2bin(command[1])
                imm_op = "1"
                ry_op = "110"
                rd_op = "0"
                d_op = "110"
                imm = val2imm(command[2])

            elif command[0] == 'load':
                pc_op = "00"
                alu_op = "01"
                rx_op = reg2bin(command[2])
                imm_op = "0"
                ry_op = "110"
                rd_op = "1"
                d_op = reg2bin(command[1])
                imm = "00000000"

            elif command[0] == 'loadi':
                pc_op = "00"
                alu_op = "11"
                rx_op = "110"
                imm_op = "1"
                ry_op = "110"
                rd_op = "1"
                d_op = reg2bin(command[1])
                imm = val2imm(command[2])

            elif command[0] == 'mov':
                pc_op = "00"
                alu_op = "01"
                rx_op = reg2bin(command[2])
                imm_op = "0"
                ry_op = "110"
                rd_op = "0"
                d_op = reg2bin(command[1])
                imm = "00000000"

            elif command[0] == 'movi':
                pc_op = "00"
                alu_op = "01"
                rx_op = "110"
                imm_op = "1"
                ry_op = "000"
                rd_op = "0"
                d_op = reg2bin(command[1])
                imm = val2imm(command[2])
           
        elif len(command) == 4:
            # add, addi, and, andi
            
            print(command)
            pc_op = "00"
            rx_op = reg2bin(command[2])
            d_op = reg2bin(command[1])
            rd_op = "0"

            if command[0] == 'add':
                alu_op = "01"
                imm_op = "0"
                ry_op = reg2bin(command[3])
                imm = "00000000"

            elif command[0] == 'addi':
                alu_op = "01"
                imm_op = "1"
                ry_op = "000"
                imm = val2imm(command[3])

            elif command[0] == 'and':
                alu_op = "00"
                imm_op = "0"
                ry_op = reg2bin(command[3])
                imm = "00000000"

            elif command[0] == 'andi':
                alu_op = "00"
                imm_op = "1"
                ry_op = "110"
                imm = val2imm(command[3])

        ans = "32'b000000" + pc_op + "00" + alu_op + "0" + rx_op + imm_op + ry_op + rd_op + d_op + imm
        hexi_1 = hex(int("0" + ans[3:], base=2))
        hexi = int(hexi_1,16)
        lista.append([ans, f"{hexi:#0{10}x}"])
    return lista

limsta = parse(d)
limsta

['movi', 'R0', '0']
['movi', 'R1', '4']
['addi', 'R0', 'R0', '1']
['and', 'R2', 'R0', 'R1']
['jz', 'R2', '2']
['movi', 'R3', '1']


[["32'b00000000000101101000000000000000", '0x00168000'],
 ["32'b00000000000101101000000100000100", '0x00168104'],
 ["32'b00000000000100001000000000000001", '0x00108001'],
 ["32'b00000000000000000001001000000000", '0x00001200'],
 ["32'b00000010000100101110011000000010", '0x0212e602'],
 ["32'b00000000000101101000001100000001", '0x00168301']]

In [100]:
file = open("wynik_parser2.txt", "w")
num = 0
for i in limsta:
    file.write(f"assign program[{num}] = " + i[0] + ';\n')
    num += 1
file.close()